# Test model example

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path

import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import shutil

import model
import evaluator
import plotting

# %matplotlib notebook
%matplotlib inline

In [ ]:
probe_type = "linear"
model_name = "hay" #'hallermann' # "hay" hay_ais
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
cell = model.create(model_name=model_name, release=True)

# probe = model.define_electrode(probe_type=probe_type)
probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]

fitness_protocols = evaluator.define_protocols(
    model_name,
    feature_set="soma",
    feature_file=f"{model_name}_model/features_list.json",
    electrode=probe,
    protocols_with_lfp=["Step1", "bAP", "BAC", "EPSP", "CaBurst"]
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

if model_name == 'hallermann':
    sim = ephys.simulators.LFPySimulator(cell, cvode_active=False, electrode=probe)
else:
    sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [ ]:
fitness_protocols
print(eva)

In [ ]:
t_start = time.time()
responses = eva.run_protocols(protocols=fitness_protocols.values(), param_values={})
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
responses

In [ ]:
plotting.plot_responses(responses)

In [ ]:
# r2={}
# r2['Step1.soma.v'] = responses['Step1.soma.v']
# plotting.plot_responses(r2)
# responses_v={}
# responses_cai={}

# for key in responses.keys():
#     if ".v" in key:
#         responses_v[key] = responses[key]
#     if ".cai" in key:
#         responses_cai[key] = responses[key] 
# plotting.plot_responses(responses_v)
# plotting.plot_responses(responses_cai)


def plot_responses(responses):
    fig, axes = plt.subplots(3, figsize=(5,5))
    for index, (resp_name, response) in enumerate(sorted(responses.items())):
        if index <6:
            axes[index].plot(response['time'], response['voltage'], label=resp_name)
            axes[index].set_title(resp_name)
        fig.tight_layout()
        fig.show()
plot_responses(responses)

In [ ]:
for resp_name, response in sorted(responses.items()):
    if 'cai' in resp_name:
        plt.plot(response['time'], response['voltage'], label=resp_name)
plt.legend()
        

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(responses['Step1.soma.v']['time'],responses['Step1.soma.v']['voltage'], label='Step1.soma.v')
# plt.plot(responses['Step1.hillock_mid.v']['time'],responses['Step1.hillock_mid.v']['voltage'], label='Step1.hillock_mid.v')
# plt.plot(responses['Step1.ais_mid.v']['time'],responses['Step1.ais_mid.v']['voltage'], label='Step1.ais_mid.v')
# plt.xlim([704,710])
# plt.legend()

for resp_name, response in sorted(responses.items()):
    if '.v' in resp_name:
        plt.plot(response['time'], response['voltage'], label=resp_name)
plt.legend()
plt.xlim([704,710])